<a href="https://colab.research.google.com/github/S3330/DL2021/blob/main/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Download the dataset
# Google Drive
!gdown --id '17Bt6c7lXt3vMRc7ZR56QWt3fXKrVtngz' --output bird.zip
!unzip -q bird.zip

Downloading...
From: https://drive.google.com/uc?id=17Bt6c7lXt3vMRc7ZR56QWt3fXKrVtngz
To: /content/bird.zip
100% 681M/681M [00:05<00:00, 120MB/s]


In [2]:
import numpy as np
import pandas as pd
import shutil, os
import matplotlib.pyplot as plt 
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import PIL
from PIL import Image
import cv2

from torch.utils.data import ConcatDataset, DataLoader, Subset
from torchvision.datasets import DatasetFolder

from tqdm.auto import tqdm

In [3]:
# create subfolders for training images
labels = pd.read_csv(
    'training_labels.txt', sep=" ",header=None)
labels.columns=['name', 'class']
labels = labels.sort_values('class')
class_names = list(labels['class'].unique())

for i in class_names:
  os.makedirs(os.path.join('train_temp', i))

for c in class_names: # category Name
  for i in list(labels[labels['class']==c]['name']): # image Id
    get_image = os.path.join('training_images/', i) # path to images
    move_image_to_cat = shutil.move(get_image, 'train_temp/'+c)

In [4]:
# spilt training and validation set
!pip install split-folders
import splitfolders
splitfolders.ratio('train_temp', output='train_and_val', seed=1337, ratio=(.8, 0.2)) 

Copying files: 3000 files [00:01, 2066.82 files/s]


In [5]:
# create a subfolder named 00 for testing images
os.makedirs(os.path.join('testing_images', '00'))
file_names = os.listdir('testing_images')

for file_name in file_names:
  if file_name != '00':
    shutil.move(os.path.join('testing_images', file_name), 'testing_images/00')

In [6]:
# delete redundant folders and rename folders
shutil.rmtree('train_temp')
shutil.rmtree('training_images')
os.rename('testing_images', 'test')

In [7]:
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    # You may add some transforms here.
    transforms.ToTensor(),
])

test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

In [8]:
batch_size = 200

# Construct datasets.
train_set = DatasetFolder('train_and_val/train', loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
valid_set = DatasetFolder('train_and_val/val', loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
test_set = DatasetFolder('test', loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)

# Construct data loaders.
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [9]:
# change output dimension to 200
import torchvision.models as models
resnet18 = models.resnet18(pretrained = True)
in_ftr  = resnet18.fc.in_features
out_ftr = 200
resnet18.fc = nn.Linear(in_ftr,out_ftr,bias=True)
# inception = models.inception_v3(pretrained = True)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [11]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize a model, and put it on the device specified.
model = resnet18.to(device)
model.device = device

# For the classification task, we use cross-entropy as the measurement of performance.
criterion = nn.CrossEntropyLoss()

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)

# The number of training epochs.
n_epochs = 10

for epoch in range(n_epochs):
    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    model.train()

    # These are used to record information in training.
    train_loss = []
    train_accs = []

    # Iterate the training set by batches.
    for batch in tqdm(train_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch

        # Forward the data. (Make sure data and model are on the same device.)
        logits = model(imgs.to(device))

        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        loss = criterion(logits, labels.to(device))

        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        # Update the parameters with computed gradients.
        optimizer.step()

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        train_loss.append(loss.item())
        train_accs.append(acc)

    # The average loss and accuracy of the training set is the average of the recorded values.
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    model.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
          logits = model(imgs.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device))

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        valid_loss.append(loss.item())
        valid_accs.append(acc)

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

    # if (i+1) == 1000:
    #   save model

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/12 [00:01<?, ?it/s]

[ Train | 001/010 ] loss = 3.32513, acc = 0.50917


  0%|          | 0/3 [00:00<?, ?it/s]

[ Valid | 001/010 ] loss = 4.27602, acc = 0.14833


...

[ Valid | 009/010 ] loss = 3.52497, acc = 0.29000


  0%|          | 0/12 [00:00<?, ?it/s]

[ Train | 010/010 ] loss = 0.02571, acc = 0.99750


  0%|          | 0/3 [00:00<?, ?it/s]

[ Valid | 010/010 ] loss = 3.51896, acc = 0.28833


In [16]:
# Make sure the model is in eval mode.
# Some modules like Dropout or BatchNorm affect if the model is in training mode.
model.eval()

# Initialize a list to store the predictions.
predictions = []

# Iterate the testing set by batches.
for batch in tqdm(test_loader):
    # A batch consists of image data and corresponding labels.
    # But here the variable "labels" is useless since we do not have the ground-truth.
    # If printing out the labels, you will find that it is always 0.
    # This is because the wrapper (DatasetFolder) returns images and labels for each batch,
    # so we have to create fake labels to make it work normally.
    imgs, labels = batch

    # We don't need gradient in testing, and we don't even have labels to compute loss.
    # Using torch.no_grad() accelerates the forward process.
    with torch.no_grad():
        logits = model(imgs.to(device))

    # Take the class with greatest logit as prediction and record it.
    predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())

  0%|          | 0/16 [00:00<?, ?it/s]

In [17]:
with open('testing_img_order.txt') as f:
  test_images = [x.strip() for x in f.readlines()]  # all the testing images
test_images.sort()
test_images

['0001.jpg',
 '0002.jpg',
 '0004.jpg',
 '0005.jpg',
 '0006.jpg',
 '0007.jpg',
 '0009.jpg',
 '0015.jpg',
 '0016.jpg',
 '0020.jpg',
 '0021.jpg',
 '0022.jpg',
 '0024.jpg',
 '0025.jpg',
 '0026.jpg',
 '0028.jpg',
 '0030.jpg',
 '0033.jpg',
 '0034.jpg',
 '0040.jpg',
 '0043.jpg',
 '0045.jpg',
 '0047.jpg',
 '0050.jpg',
 '0051.jpg',
 '0052.jpg',
 '0053.jpg',
 '0054.jpg',
 '0055.jpg',
 '0059.jpg',
 '0060.jpg',
 '0064.jpg',
 '0071.jpg',
 '0072.jpg',
 '0074.jpg',
 '0075.jpg',
 '0076.jpg',
 '0077.jpg',
 '0078.jpg',
 '0080.jpg',
 '0081.jpg',
 '0085.jpg',
 '0088.jpg',
 '0091.jpg',
 '0092.jpg',
 '0095.jpg',
 '0097.jpg',
 '0100.jpg',
 '0103.jpg',
 '0104.jpg',
 '0105.jpg',
 '0107.jpg',
 '0108.jpg',
 '0110.jpg',
 '0113.jpg',
 '0114.jpg',
 '0115.jpg',
 '0116.jpg',
 '0120.jpg',
 '0122.jpg',
 '0125.jpg',
 '0126.jpg',
 '0127.jpg',
 '0129.jpg',
 '0130.jpg',
 '0135.jpg',
 '0136.jpg',
 '0138.jpg',
 '0143.jpg',
 '0147.jpg',
 '0151.jpg',
 '0154.jpg',
 '0155.jpg',
 '0156.jpg',
 '0157.jpg',
 '0158.jpg',
 '0159.jpg',

In [18]:
num = list(range(0,200))
num_to_class_dict = dict(zip(num, class_names))
num_to_class_dict

{0: '001.Black_footed_Albatross',
 1: '002.Laysan_Albatross',
 2: '003.Sooty_Albatross',
 3: '004.Groove_billed_Ani',
 4: '005.Crested_Auklet',
 5: '006.Least_Auklet',
 6: '007.Parakeet_Auklet',
 7: '008.Rhinoceros_Auklet',
 8: '009.Brewer_Blackbird',
 9: '010.Red_winged_Blackbird',
 10: '011.Rusty_Blackbird',
 11: '012.Yellow_headed_Blackbird',
 12: '013.Bobolink',
 13: '014.Indigo_Bunting',
 14: '015.Lazuli_Bunting',
 15: '016.Painted_Bunting',
 16: '017.Cardinal',
 17: '018.Spotted_Catbird',
 18: '019.Gray_Catbird',
 19: '020.Yellow_breasted_Chat',
 20: '021.Eastern_Towhee',
 21: '022.Chuck_will_Widow',
 22: '023.Brandt_Cormorant',
 23: '024.Red_faced_Cormorant',
 24: '025.Pelagic_Cormorant',
 25: '026.Bronzed_Cowbird',
 26: '027.Shiny_Cowbird',
 27: '028.Brown_Creeper',
 28: '029.American_Crow',
 29: '030.Fish_Crow',
 30: '031.Black_billed_Cuckoo',
 31: '032.Mangrove_Cuckoo',
 32: '033.Yellow_billed_Cuckoo',
 33: '034.Gray_crowned_Rosy_Finch',
 34: '035.Purple_Finch',
 35: '036.Nor

In [19]:
predicted_class = []
for i in range(len(predictions)):
  predicted_class.append(num_to_class_dict[predictions[i]])
predicted_class

['085.Horned_Lark',
 '134.Cape_Glossy_Starling',
 '042.Vermilion_Flycatcher',
 '081.Pied_Kingfisher',
 '112.Great_Grey_Shrike',
 '118.House_Sparrow',
 '123.Henslow_Sparrow',
 '176.Prairie_Warbler',
 '130.Tree_Sparrow',
 '197.Marsh_Wren',
 '039.Least_Flycatcher',
 '106.Horned_Puffin',
 '134.Cape_Glossy_Starling',
 '124.Le_Conte_Sparrow',
 '177.Prothonotary_Warbler',
 '199.Winter_Wren',
 '075.Green_Jay',
 '070.Green_Violetear',
 '159.Black_and_white_Warbler',
 '092.Nighthawk',
 '019.Gray_Catbird',
 '024.Red_faced_Cormorant',
 '054.Blue_Grosbeak',
 '054.Blue_Grosbeak',
 '191.Red_headed_Woodpecker',
 '085.Horned_Lark',
 '112.Great_Grey_Shrike',
 '010.Red_winged_Blackbird',
 '068.Ruby_throated_Hummingbird',
 '134.Cape_Glossy_Starling',
 '039.Least_Flycatcher',
 '082.Ringed_Kingfisher',
 '023.Brandt_Cormorant',
 '151.Black_capped_Vireo',
 '133.White_throated_Sparrow',
 '168.Kentucky_Warbler',
 '164.Cerulean_Warbler',
 '078.Gray_Kingbird',
 '065.Slaty_backed_Gull',
 '152.Blue_headed_Vireo',
 

In [20]:
search_dict = dict(zip(test_images, predicted_class))
search_dict

{'0001.jpg': '085.Horned_Lark',
 '0002.jpg': '134.Cape_Glossy_Starling',
 '0004.jpg': '042.Vermilion_Flycatcher',
 '0005.jpg': '081.Pied_Kingfisher',
 '0006.jpg': '112.Great_Grey_Shrike',
 '0007.jpg': '118.House_Sparrow',
 '0009.jpg': '123.Henslow_Sparrow',
 '0015.jpg': '176.Prairie_Warbler',
 '0016.jpg': '130.Tree_Sparrow',
 '0020.jpg': '197.Marsh_Wren',
 '0021.jpg': '039.Least_Flycatcher',
 '0022.jpg': '106.Horned_Puffin',
 '0024.jpg': '134.Cape_Glossy_Starling',
 '0025.jpg': '124.Le_Conte_Sparrow',
 '0026.jpg': '177.Prothonotary_Warbler',
 '0028.jpg': '199.Winter_Wren',
 '0030.jpg': '075.Green_Jay',
 '0033.jpg': '070.Green_Violetear',
 '0034.jpg': '159.Black_and_white_Warbler',
 '0040.jpg': '092.Nighthawk',
 '0043.jpg': '019.Gray_Catbird',
 '0045.jpg': '024.Red_faced_Cormorant',
 '0047.jpg': '054.Blue_Grosbeak',
 '0050.jpg': '054.Blue_Grosbeak',
 '0051.jpg': '191.Red_headed_Woodpecker',
 '0052.jpg': '085.Horned_Lark',
 '0053.jpg': '112.Great_Grey_Shrike',
 '0054.jpg': '010.Red_winge

In [21]:
submission = []
with open('testing_img_order.txt') as f:
  correct_order = [x.strip() for x in f.readlines()]
for i in tqdm(correct_order):
  predicted_class = search_dict[i]  # the predicted category
  submission.append([i, predicted_class])
np.savetxt('answer.txt', submission, fmt='%s')

  0%|          | 0/3033 [00:00<?, ?it/s]

In [23]:
torch.save(model, './mymodel.pt')